In [1]:
!pip install -q -U sentence-transformers accelerate transformers bitsandbytes \
    --prefer-binary --extra-index-url https://download.pytorch.org/whl/cu118 chromadb

import nltk

nltk.download('punkt_tab')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 835.3 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 38.1 MB/s et

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [87]:
import pandas as pd
import json

# Set LOCAL variable values
# Article (data) selection
main_data = json.load(open("wikinews_data.json", 'r'))
for i, title in enumerate([d['title'] for d in main_data]):
  print(f'{i}: {title}')

# Change the index for new article
ARTICLE_INDEX = 14
data = main_data[ARTICLE_INDEX]
print(f"\nWill be working with '{data['title']}' article data!")

source_data = data['source_data']
chroma_collection_name = data['chroma_collection_name']
chroma_db_path = "chroma_db"
model_path = "models"
print(f"\nStarted with the plain LLM & RAG-enhanced LLM generation of '{data['title']}' article")

0: US: Tulsa residents approve $814 million infrastructure package
1: 78th British Academy Film Awards held in London
2: Ryan Gosling cast in upcoming Star Wars film
3: Thai officials seize 238 tons of illegal e-waste at Bangkok port
4: 20-year-old astrophotographer captures rare solar eclipse on Saturn
5: Scientists discover seagrass off Australia is world's largest plant
6: India defeats New Zealand to win 2025 Champions Trophy
7: Researchers film colossal squid in its natural habitat for the first time
8: SpaceX will return stranded astronauts in February 2025, NASA announces
9: Microsoft, Nware sign 10-year cloud gaming deal
10: United Kingdom buries Queen Elizabeth II after state funeral
11: UK heavy metal band Black Sabbath announces final performance with original lineup
12: GSK rejects three Unilever bids to buy consumer healthcare arm, says unit was 'fundamentally undervalued'
13: FIFA World Cup 2018 Last 16: France, Uruguay send Argentina, Portugal home
14: European Union to 

In [4]:
from huggingface_hub import login

# Hugging Face login
token = open("hf_token.txt", "r").read().strip()
login(token=token, add_to_git_credential=True)
print("Logged in to Hugging Face and the access token is cached!")

Logged in to Hugging Face and the access token is cached!


In [58]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch, os

# Load LLM
# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model name and local path
llm_name = "mistral-7b-instruct"
local_dir = f"{model_path}/{llm_name}"

# If model isn't already downloaded, fetch and save to local directory
if not os.path.exists(local_dir):
    print(f"Downloading {llm_name} to {local_dir} ...")
    tokenizer = AutoTokenizer.from_pretrained(llm_name)
    model = AutoModelForCausalLM.from_pretrained(
        llm_name,
        device_map="auto",
        torch_dtype=torch.float16
    )
    tokenizer.save_pretrained(local_dir)
    model.save_pretrained(local_dir)
    print("Model download complete and saved locally!")
else:
    print(f"Model already downloaded in {local_dir}. Loading from local path.")
    tokenizer = AutoTokenizer.from_pretrained(local_dir)
    model = AutoModelForCausalLM.from_pretrained(
        local_dir,
        device_map="auto",
        torch_dtype=torch.float16,
    )

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Create generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

print(f"Loaded model from {local_dir} successfully on {device.upper()}!")

Model already downloaded in /content/drive/MyDrive/Colab Notebooks/Dissertation/models/mistral-7b-instruct. Loading from local path.


Device set to use cuda:0


Loaded model from /content/drive/MyDrive/Colab Notebooks/Dissertation/models/mistral-7b-instruct successfully on CUDA!


In [88]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [89]:
import textwrap
import re

# Count number of words and tokens in the original Human article
article_text = data['article_text']

print("HUMAN ARTICLE -- ")
print(data['title'])
wrapper = lambda text: textwrap.fill(text, width=100)
print(wrapper(article_text))

human_article_words_list = re.findall(r"\b\w+\b", article_text)
print(f"Word Count: {len(human_article_words_list)}")

human_article_tokens_list = tokenizer.encode(article_text, truncation=False, add_special_tokens=False)
print(f"Token Count: {len(human_article_tokens_list)}")

HUMAN ARTICLE -- 
European Union to reduce carbon emissions by 55% of 1990 levels by 2030
On Wednesday, the European Union (EU) set a new goal to cut carbon emissions by at least 55% from
1990 levels by 2030 and reach zero net emissions by 2050, increasing the reduction target set in
2014, which was 40% by 2030 and 80-95% by 2050.The EU Climate Law, agreed after 14 hours of
negotiations includes the establishment of a 15-member independent body to advise on proposed
climate policies and increased investment in 'carbon sinks'. According to members of the European
Parliament (MEP) those will ensure 57% net reduction target is in place. This claim has been
disputed by senior policy officer for climate and energy at the European Environmental Bureau
Barbara Mariani, who says the measures equal a 52.8% cut in actual emissions.A September 2020 press
release by the European Commission (EC) promised 'following broad public consultation and thorough
impact assessments, the Commission will come 

In [90]:
from nltk.tokenize import sent_tokenize


def estimate_context_budget(word_bin):
    # Bin ceilings
    if word_bin == "Short":
        cap = 1400
    elif word_bin == "Medium":
        cap = 2200
    else:
        cap = 3000

    return cap


reference_chunks = [s.strip() for s in sent_tokenize(source_data) if s.strip()]

# de-dupe while preserving order
seen = set()
reference_chunks = [c for c in reference_chunks if not (c in seen or seen.add(c))]

MAX_CONTEXT_TOKEN_LIMIT = estimate_context_budget(data['word_count_bin'])

sampled, tok_count = [], 0

# First n approach
for chunk in reference_chunks:
    tlen = len(tokenizer.encode(chunk, add_special_tokens=False))
    if tok_count + tlen > MAX_CONTEXT_TOKEN_LIMIT:
        break
    sampled.append(chunk)
    tok_count += tlen

context_text = "\n".join(sampled)
context_tokens = tokenizer.encode(
    context_text,
    truncation=True,
    max_length=MAX_CONTEXT_TOKEN_LIMIT,
    add_special_tokens=False
    )
print(f"Token Count (Plain LLM context): {len(context_tokens)}")
context_text = tokenizer.decode(context_tokens)

print(f"Sample context Text: {context_text[:100]}")

Token Count (Plain LLM context): 3000
Sample context Text: The EU has adopted ambitious new targets to curb climate change, with a pledge to make them legally 


In [91]:
# Prompt Design
prompt = f"""
You are a Wikinews journalist. Based ONLY on the CONTEXT below, write a complete, high-quality news article.

### CONTEXT
{context_text}

### ARTICLE RULES
- The article's body must be between {len(human_article_words_list) + 1} and {int(len(human_article_words_list) * 1.10)} words long.
- If you’re close to the upper bound, finish the current sentence and stop.
- End with ONE clear concluding sentence, then on a new line output exactly: <END>
- Do not output anything after <END>.
- Start with the Article Title: (Use exactly as given) {data['title']}.
- Always use past tense for writing the article.
- Write a lead summary: a short paragraph capturing who, what, when, where.
- Follow it with 2–3 paragraphs with details, using the CONTEXT only.
- Use the inverted pyramid style: most important and newsworthy facts first, less immediate background details later.
- Use neutral tone no opinions, no bullet points and no Q&A format.
- Do not make up anything not supported by the CONTEXT.
- Do not repeat the same fact; if similar sentences appear in the CONTEXT, consolidate them once.
- Do not add headings or extra sections.
- Ensure spelling and grammar are correct and consistent.
- Use professional journalistic English.
- Ignore any bracketed meta lines like [If you like this prompt...] and do NOT include them in the article.
- Do not include any disclaimers, model metadata, license text, or statements like 'This article was generated'. Output only the article text.

[Start the article below]
"""

# Tokenize with padding and attention mask
input_data = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True
).to(model.device)

# Set pad_token_id
pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

# Generate
output_ids = model.generate(
    input_ids=input_data["input_ids"],
    attention_mask=input_data["attention_mask"],
    # min_new_tokens = int(len(human_article_tokens_list) * 1.02),
    # max_new_tokens = int(len(human_article_tokens_list) * 1.20)
    max_new_tokens= int(len(human_article_words_list) * 1.45) + 70,
    do_sample=False,
    pad_token_id=pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    # num_beams=4,
    # early_stopping=True
)

# Decode
plain_llm_version = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Remove the prompt and the end of article placeholders from the LLM response
for unwanted_string in [prompt, '[End of article]', '<END_ARTICLE>', '<END>']:
  plain_llm_version = plain_llm_version.replace(unwanted_string, "")

print("PLAIN LLM ARTICLE -- ")
print(wrapper(plain_llm_version))
words = plain_llm_version.split()
print(f"Word Count: {len(words)}")
tokens = tokenizer.encode(plain_llm_version, truncation=False, add_special_tokens=False)
print(f"Token Count: {len(tokens)}")

# Save
filename = f"{data['title']}_plain_LLM_article_V3.txt".replace(" ", "_")
with open(f"/content/drive/MyDrive/Colab Notebooks/Dissertation/Article_{ARTICLE_INDEX}/{filename}", "w", encoding="utf-8") as f:
    f.write(plain_llm_version)
data['plain_llm_article'] = plain_llm_version
print(f"Plain LLM article saved to /content/drive/MyDrive/Colab Notebooks/Dissertation/Article_{ARTICLE_INDEX}/{filename}!")

PLAIN LLM ARTICLE -- 
 European Union to reduce carbon emissions by 55% of 1990 levels by 2030  The European Union (EU)
has agreed on ambitious new targets to curb climate change, with a pledge to make them legally
binding. Under a new law agreed between member states and the EU Parliament, the bloc will cut
carbon emissions by at least 55% by 2030, compared with 1990 levels.  The EU Parliament had pushed
for a higher target of a 60% reduction. EU Commission chief Ursula von der Leyen said, "Our
political commitment to becoming the first climate neutral continent by 2050 is now also a legal
one." The deal comes ahead of a virtual summit of world leaders later this week, where the US is
expected to announce its own climate targets for 2030.  US President Joe Biden, who will lead the
meeting, rejoined the Paris climate agreement in his first day in office and has previously
committed to reaching net zero emissions by 2050. The UK, meanwhile, announced radical plans to cut
carbon emission

In [21]:
import torch
from sentence_transformers import SentenceTransformer

# Load embedding model
embedding_model_path = f"{model_path}/bge-large-en-v1.5"
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = SentenceTransformer(embedding_model_path, device=device)
embedding_model.max_seq_length = 512

In [92]:
from chromadb import PersistentClient

# Load ChromaDB collection
client = PersistentClient(path=chroma_db_path)
collection = client.get_collection(name=chroma_collection_name)

# Choose candidate pool by bin
if data['word_count_bin'] == "Long":
    N_RESULTS = 120
elif data['word_count_bin'] == "Medium":
    N_RESULTS = 100
else:
    N_RESULTS = 70

# Encode first query (title)
query = data['title']
q1 = embedding_model.encode("query: " + query, normalize_embeddings=True)

# Encode second query (title + "key facts")
q2 = embedding_model.encode("query: " + (query + " key facts and details"), normalize_embeddings=True)

# q1, q2 Queries for Retrieval
res1 = collection.query(query_embeddings=[q1.tolist()], n_results=N_RESULTS, include=["documents", "distances"])
res2 = collection.query(query_embeddings=[q2.tolist()], n_results=N_RESULTS, include=["documents", "distances"])

# Retrieve top-N documents for both queries
res1 = collection.query(query_embeddings=[q1.tolist()], n_results=50, include=["documents", "distances"])
res2 = collection.query(query_embeddings=[q2.tolist()], n_results=50, include=["documents", "distances"])

ids1 = res1.get("ids", [[]])[0]
ids2 = res2.get("ids", [[]])[0]

docs1, dists1 = res1["documents"][0], res1["distances"][0]
docs2, dists2 = res2["documents"][0], res2["distances"][0]

# Merge: keep highest-sim per id
merged = {}
for _id, doc, dist in zip(ids1, docs1, dists1):
    sim = 1 - dist
    if (_id not in merged) or (sim > merged[_id]["sim"]):
        merged[_id] = {"doc": doc, "sim": sim}

for _id, doc, dist in zip(ids2, docs2, dists2):
    sim = 1 - dist
    if (_id not in merged) or (sim > merged[_id]["sim"]):
        merged[_id] = {"doc": doc, "sim": sim}

# Sort and select unique top-N
pairs = sorted(((id_, v["sim"], v["doc"]) for id_, v in merged.items()), key=lambda x: x[1], reverse=True)

# Build up to the SAME adaptive budget as plain LLM + 100 tokens to compensate for passage: in ach and every embedding
MAX_CONTEXT_TOKEN_LIMIT = estimate_context_budget(data['word_count_bin']) + 100

sampled_docs, tok_count = [], 0
for _, _, doc in pairs:
    tlen = len(tokenizer.encode(doc, add_special_tokens=False))
    if tok_count + tlen > MAX_CONTEXT_TOKEN_LIMIT:
        break
    sampled_docs.append(doc)
    tok_count += tlen

context_text = "\n".join(sampled_docs)

# Print with ids and similarities
print("Selected top unique context chunks:")
for rank, (id_, sim, doc) in enumerate(pairs):
    print(f"{rank}. id={id_}, similarity={sim}")

# Truncate to stay within model token limit
context_tokens = tokenizer.encode(
    context_text,
    truncation=True,
    max_length=MAX_CONTEXT_TOKEN_LIMIT,
    add_special_tokens=False
    )

print(f"Token Count (RAG context): {len(context_tokens)}")
context_text = tokenizer.decode(context_tokens)

Selected top unique context chunks:
0. id=1, similarity=0.8093758821487427
1. id=12, similarity=0.8036229014396667
2. id=21, similarity=0.792255699634552
3. id=0, similarity=0.7887791395187378
4. id=22, similarity=0.7875897288322449
5. id=30, similarity=0.7846798300743103
6. id=89, similarity=0.7845910787582397
7. id=114, similarity=0.7830570340156555
8. id=16, similarity=0.7824594974517822
9. id=35, similarity=0.7815335392951965
10. id=143, similarity=0.7783237099647522
11. id=36, similarity=0.7778615355491638
12. id=29, similarity=0.7770565152168274
13. id=11, similarity=0.7770453691482544
14. id=115, similarity=0.7738066911697388
15. id=34, similarity=0.7728836536407471
16. id=112, similarity=0.7728740572929382
17. id=98, similarity=0.7713354229927063
18. id=20, similarity=0.7675957679748535
19. id=19, similarity=0.7674711346626282
20. id=23, similarity=0.766655445098877
21. id=7, similarity=0.7665737271308899
22. id=107, similarity=0.7651887536048889
23. id=88, similarity=0.7644560

In [93]:
# Prompt Design
prompt = f"""
You are a Wikinews journalist. Based ONLY on the CONTEXT below, write a complete, high-quality news article.

### CONTEXT
{context_text}

### ARTICLE RULES
- The article's body must be between {len(human_article_words_list) + 1} and {int(len(human_article_words_list) * 1.10)} words long.
- If you’re close to the upper bound, finish the current sentence and stop.
- End with ONE clear concluding sentence, then on a new line output exactly: <END>
- Do not output anything after <END>.
- Start with the Article Title: (Use exactly as given) {data['title']}.
- Always use past tense for writing the article.
- Write a lead summary: a short paragraph capturing who, what, when, where.
- Follow it with 2–3 paragraphs with details, using the CONTEXT only.
- Use the inverted pyramid style: most important and newsworthy facts first, less immediate background details later.
- Use neutral tone no opinions, no bullet points and no Q&A format.
- Do not make up anything not supported by the CONTEXT.
- Do not repeat the same fact; if similar sentences appear in the CONTEXT, consolidate them once.
- Do not add headings or extra sections.
- Ensure spelling and grammar are correct and consistent.
- Use professional journalistic English.
- Ignore any bracketed meta lines like [If you like this prompt...] and do NOT include them in the article.
- Do not include any disclaimers, model metadata, license text, or statements like 'This article was generated'. Output only the article text.

[Start the article below]
"""

# Tokenize input
input_data = tokenizer(prompt,
                       return_tensors="pt",
                       truncation=True).to(model.device)

# Set PAD token_id
pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

# Generate article
output_ids = model.generate(
    input_ids=input_data["input_ids"],
    attention_mask=input_data["attention_mask"],
    # min_new_tokens = int(len(human_article_tokens_list) * 1.02),
    # max_new_tokens = int(len(human_article_tokens_list) * 1.20)
    max_new_tokens= int(len(human_article_words_list) * 1.45) + 70,
    do_sample=False,
    # no_repeat_ngram_size=3,
    # repetition_penalty=1.05,
    pad_token_id=pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
rag_version = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Remove the prompt and the end of article placeholders from the LLM response
for unwanted_string in [prompt, '[End of article]', '<END_ARTICLE>', '<END>']:
  rag_version = rag_version.replace(unwanted_string, "")

print("RAG-ENHANCED LLM ARTICLE -- ")
print(wrapper(rag_version))
words = rag_version.split()
print(f"Word Count: {len(words)}")
tokens = tokenizer.encode(rag_version, truncation=False, add_special_tokens=False)
print(f"Token Count: {len(tokens)}")

# Save
filename = f"{data['title']}_rag_enhanced_article_V3.txt".replace(" ", "_")
with open(f"/content/drive/MyDrive/Colab Notebooks/Dissertation/Article_{ARTICLE_INDEX}/{filename}", "w", encoding="utf-8") as f:
    f.write(rag_version)
data['rag_llm_article'] = rag_version
print(f"RAG-enhanced article saved to /content/drive/MyDrive/Colab Notebooks/Dissertation/Article_{ARTICLE_INDEX}/{filename}")

main_data[ARTICLE_INDEX] = data
json.dump(main_data, open("wikinews_data.json", "w"), indent=3, force_ascii=False, orient="records")

RAG-ENHANCED LLM ARTICLE -- 
 European Union to reduce carbon emissions by 55% of 1990 levels by 2030  The European Union (EU)
has agreed on a new law to cut carbon emissions by at least 55% by 2030, compared with 1990 levels.
The decision was reached after 14 hours of intensive negotiations between member states and the EU
Parliament.  The new target, which was initially announced by EU leaders in December, is a
significant increase from the previous 40% reduction target. The development is seen as a key
milestone on the road to the United Nations climate change conference (COP26) in Glasgow this
November, where leaders will underscore the need to limit planetary warming to 1.5°C by the end of
the century.  Under the new law, the EU will invest further in 'carbon sinks' such as forests,
grasslands, and wetlands. MEPs involved in the talks claimed this would in effect equate to a 57%
net reduction target for member states. The annual reduction in the 'cap' on emissions from EU
Emission